In [1]:
%cd /content/drive/MyDrive/#temp/ai_pu_yun
!pip install -r requirements.txt
#!unzip arrays.zip

/content/drive/MyDrive/experience_job_application/companies-interviews/ai_pu_yun
     |████████████████████████████████| 516.2 MB 5.3 kB/s 
     |████████████████████████████████| 1.0 MB 58.0 MB/s 
     |████████████████████████████████| 454 kB 63.7 MB/s 
     |████████████████████████████████| 3.0 MB 64.7 MB/s 
     |████████████████████████████████| 2.9 MB 58.6 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing 

Testing Vision Transformer

In [3]:
# module import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os, sys, glob
import PIL
#from imutils.video import FPS
from queue import Queue
from datetime import datetime

# sklearn packages
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
"""
# torch packages
import torch
import torchvision
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Running on {device}')

"""
# tf packages
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import tensorflow_addons as tfa
# tensorboard initializations
"""
%load_ext tensorboard
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=0)
"""
# test gpu 
print("CUDA is running.") if tf.test.is_gpu_available() else print("CUDA is not running.")
#"""

from vit import VisionTransformer


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA is not running.


In [ ]:
with open('.vit_weights_code.txt','r+') as f:
    VIT_WEIGHTS_DIR = f.readline()
with open('.vit_model_code.txt','r+') as f:
    VIT_MODEL_DIR = f.readline()


In [ ]:
# choose one and comment the other to try
VIT_DIR = VIT_WEIGHTS_DIR 
#VIT_DIR = VIT_MODEL_DIR

In [4]:
IMAGE_SIZE=256
PATCH_SIZE=7
LAYERS=8
D_MODEL=64
NUM_HEADS=4
MLP_DIM=128
LR=3e-4
WEIGHT_DECAY=1e-3
EPOCHS=40

def create_model():
    model = VisionTransformer(
        image_size=IMAGE_SIZE,
        patch_size=PATCH_SIZE,
        num_layers=LAYERS,
        num_classes=2,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        mlp_dim=MLP_DIM,
        channels=3,
        dropout=0.1,
    )

    model.compile(
        loss='binary_crossentropy',
        optimizer=tfa.optimizers.AdamW(
            learning_rate=LR, weight_decay=WEIGHT_DECAY
        ),
        metrics=["accuracy"],
    )
    return model

model = create_model()

#model.load_weights('logs/weights/vit') # only pass the prefix
model.load_weights(VIT_DIR) # only pass the prefix


In [6]:
"""
RUN THIS BEFORE TRAINING STARTS TO UPDATE MODEL CODENAME
"""
timenow = datetime.now().strftime("%m%d %H:%M:%S").split(' ')
datecode,timecode = timenow[0], ''
for i, s in enumerate(timenow[1].split(':')): 
    if i == 0: s = str(int(s)+8) #taiwan time
    timecode += s
code = datecode + '_' + timecode

print("This session's code:", code)

This session's code: 1109_202011


In [7]:
TEST_DIR = 'test_set'
test_set = glob.glob("test_set/*/*")

images, preds, truths = [], [], []

print(test_set)

for i, data in enumerate(test_set):
    label = 1 if data.split('/')[2] == 'df' else 0
    truths.append(label)
    
    # predict
    img = cv2.imread(data)[:,:,::-1]
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    images.append(img)

images = np.array(images).reshape(-1,256,256,3)
#print(images)
truths = to_categorical(truths)

results = model.evaluate(images, truths)

print(results)

['test_set/df/114_642.jpg', 'test_set/df/114_714.jpg', 'test_set/df/114_666.jpg', 'test_set/df/114_54.jpg', 'test_set/df/115_84.jpg', 'test_set/df/115_180.jpg', 'test_set/df/115_150.jpg', 'test_set/df/116_480.jpg', 'test_set/df/116_352.jpg', 'test_set/df/116_424.jpg', 'test_set/df/117_246.jpg', 'test_set/df/117_132.jpg', 'test_set/df/117_156.jpg', 'test_set/df/118_108.jpg', 'test_set/df/118_87.jpg', 'test_set/real/34_24.jpg', 'test_set/real/34_6.jpg', 'test_set/real/34_0.jpg', 'test_set/real/37_318.jpg', 'test_set/real/37_12.jpg', 'test_set/real/37_18.jpg', 'test_set/real/38_162.jpg', 'test_set/real/38_60.jpg', 'test_set/real/38_90.jpg', 'test_set/real/39_18.jpg', 'test_set/real/39_66.jpg', 'test_set/real/39_12.jpg', 'test_set/real/40_138.jpg', 'test_set/real/40_150.jpg', 'test_set/real/40_132.jpg']
1/1 [==============================] - 0s 2ms/step - loss: 0.7199 - accuracy: 1.0000
[0.719942033290863, 1.0]


In [42]:
"""
Test on video
"""

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import cv2
import os, sys, glob

labels, faces = [], []

test_videos = glob.glob('test_data/*.mp4')
print(test_videos)
IMAGE_SHAPE = (256,256,3)
CASCADE_FILE = 'cascade_file/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(CASCADE_FILE)

for video in test_videos:
    cnt = 0
    cap = cv2.VideoCapture(video)

    if not cap.isOpened():
        print('Error reading test video')
    
    while cnt != 500 and cap.isOpened(): # detect first 500 frames for convenience
        ret, frame = cap.read()
        frame = frame[:,:,::-1] # convert to RGB
        frame = cv2.resize(frame, (256,256), interpolation=cv2.INTER_AREA)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # detece face
        face = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(50, 50),
        )
        if cnt%10==0 and len(face)>0:
            # if face detected
            for x, y, w, h in face:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
                face = frame[y:y + h, x:x + w]
                face = cv2.resize(face, (256,256), interpolation=cv2.INTER_CUBIC)

            faces.append(face)
            print(video)
            label = 1. if video.split('-')[1] == 'df' else 0. #get label
            labels.append(label) 
            plt.axis('off')
            plt.imshow(face)
            plt.show()
        
        cnt += 1
cap.release()


Output hidden; open in https://colab.research.google.com to view.

In [45]:
faces = np.array(faces)
labels = np.array(labels)

In [11]:
#pred = model.predict(faces)
model.evaluate(faces, labels)

### EfficientNet

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import shutil
import os, random, sys
import datetime
import csv
from pylab import *
import itertools
from PIL import Image
import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, models, layers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
#!pip install efficientnet
from efficientnet.tfkeras import EfficientNetB0 
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import load_model

with open('.effnet_code.txt','r+') as f:
    EFFNET_DIR = f.readline()

model = load_model(EFFNET_DIR)
#model = load_model('models/effnet_1109_202124/effnet_1109_202124.h5')
eff_pred = model.predict(faces)

In [49]:
preds = []
for pr in pred:
    verdict = np.argmax(pr)
    preds.append(verdict)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for i in range(len(faces)):
    im = faces[i]
    plt.subplot(len(labels)/10+1,10,i+1)
    plt.imshow(im)
    color = 'green' if preds[i] == labels[i] else 'red'
    plt.text(im.shape[1]/2,0,f'{labels[i]}\n', ha="center", va="bottom", size="medium")
    plt.text(im.shape[1]/2,0,f'\n{preds[i]}', ha="center", va="bottom", size="medium",color=color)
    plt.axis('off')
